In [2]:
import json
import pandas as pd
import numpy as np
from pandas import DataFrame
import glob

WIND_SCALER_BEST=1
WIND_SCALER_WORST=1
SOLAR_SCALER_BEST=1
SOLAR_SCALER_WORST=1

AIR_DENSITY=1.225
TURBINE_EFF=0.4
SOLAR_PANEL_EFF=0.18
PERF_RATE=0.75
GAMMA=0.0045
WIND_CITIES= {
    "Izmir":0.38,
    "Balikesir":0.285,
    "Canakkale":0.18,
    "Manisa":0.155
}
SOLAR_CITIES={
    "Konya":0.44,
    "Ankara":0.2,
    "Sanliurfa":0.19,
    "Kayseri":0.17
}

In [4]:
all_files=glob.glob("WEATHER JSONS/*_weather.json")
li=[]
for filename in all_files:
    with open(filename,'r',encoding="utf-8") as json_file:
        data = json.load(json_file)
        temp_df=pd.json_normalize(data,record_path=['days','hours'],meta=[['days','datetime'],['address']])
        li.append(temp_df)
df_combined=pd.concat(li,axis=0,ignore_index=True)

In [207]:
df_energy=pd.read_json("energy_data.json")
df_energy['DATE/HOUR'] = pd.to_datetime(
    df_energy['Date'].astype(str) + ' ' + df_energy['Hour'].astype(str)+':00'
)
deleted_columns=["Date","Hour"]
df_energy.drop(columns=deleted_columns,inplace=True)
df_energy.set_index(df_energy['DATE/HOUR'],inplace=True)

In [208]:
df_energy['Wind Energy']=df_energy['Wind Energy'].astype(str).str.strip()
df_energy["Wind Energy"] = df_energy["Wind Energy"].str.replace('.','')
df_energy["Wind Energy"] = df_energy["Wind Energy"].str.replace(',','.')
df_energy["Wind Energy"] = df_energy["Wind Energy"].astype(float)
df_energy

,Total Energy,Solar Energy,Wind Energy,DATE/HOUR
DATE/HOUR,,,,
2025-08-15 00:00:00,"46.930,69","0,01",6270.54,2025-08-15 00:00:00
2025-08-15 01:00:00,"44.939,70","0,01",6039.83,2025-08-15 01:00:00
2025-08-15 02:00:00,"43.132,34","0,01",6061.29,2025-08-15 02:00:00
2025-08-15 03:00:00,"41.894,37","0,01",6079.25,2025-08-15 03:00:00
2025-08-15 04:00:00,"40.912,47","0,92",6241.11,2025-08-15 04:00:00
...,...,...,...,...
2025-11-15 19:00:00,"42.332,06","2,64",2331.31,2025-11-15 19:00:00
2025-11-15 20:00:00,"41.332,64","2,64",2282.53,2025-11-15 20:00:00
2025-11-15 21:00:00,"40.344,24","2,76",2107.14,2025-11-15 21:00:00


In [209]:
target_colunms=['days.datetime','datetime','temp','solarradiation','windspeed','address']
df_final=df_combined[target_colunms].copy()

In [210]:
def convert_units(df1: DataFrame) -> DataFrame:
    #F TO C
    df1['temp_c']=(df1['temp']-32)*5/9
    #MPH TO M/S
    df1['windspeed_ms']=df1['windspeed']*0.44704
    #merging datatime day-hour
    df1['Full Date']=pd.to_datetime(df1['days.datetime']+' '+df1['datetime'])
    deleted_columns=['days.datetime','datetime','windspeed','temp']
    df1.drop(deleted_columns,inplace=True,axis=1)
    return df1
df_final=convert_units(df_final)

In [211]:
def handling_wind_sensor_anomalies(df: DataFrame,window_size) -> DataFrame:
    std_rolling=df['windspeed_ms'].rolling(window=window_size).std()
    is_stuck=(std_rolling==0)&(df['windspeed_ms']>0)
    df['windspeed_ms']=np.where(is_stuck,0,df['windspeed_ms'])
    return df
handling_wind_sensor_anomalies(df_final,6)

,solarradiation,address,temp_c,windspeed_ms,Full Date
0,0.0,izmir,23.388889,3.039872,2025-08-15 00:00:00
1,0.0,izmir,26.222222,7.152640,2025-08-15 01:00:00
2,0.0,izmir,21.777778,3.039872,2025-08-15 02:00:00
3,0.0,izmir,22.000000,6.795008,2025-08-15 03:00:00
4,0.0,izmir,29.444444,1.922272,2025-08-15 04:00:00
...,...,...,...,...,...
3187,0.0,Konya,8.944444,3.084576,2025-11-15 19:00:00
3188,0.0,Konya,6.944444,2.592832,2025-11-15 20:00:00
3189,0.0,Konya,6.944444,3.084576,2025-11-15 21:00:00
3190,0.0,Konya,6.944444,1.519936,2025-11-15 22:00:00


In [212]:
df_final

,solarradiation,address,temp_c,windspeed_ms,Full Date
0,0.0,izmir,23.388889,3.039872,2025-08-15 00:00:00
1,0.0,izmir,26.222222,7.152640,2025-08-15 01:00:00
2,0.0,izmir,21.777778,3.039872,2025-08-15 02:00:00
3,0.0,izmir,22.000000,6.795008,2025-08-15 03:00:00
4,0.0,izmir,29.444444,1.922272,2025-08-15 04:00:00
...,...,...,...,...,...
3187,0.0,Konya,8.944444,3.084576,2025-11-15 19:00:00
3188,0.0,Konya,6.944444,2.592832,2025-11-15 20:00:00
3189,0.0,Konya,6.944444,3.084576,2025-11-15 21:00:00
3190,0.0,Konya,6.944444,1.519936,2025-11-15 22:00:00


In [213]:
#Matematical Calculations
#RAW WIND ENERGY
def raw_wind_energy_cal(df: DataFrame) -> None:
    vel=df['windspeed_ms']
    calculation_part=(vel**3)*AIR_DENSITY*TURBINE_EFF
    df['RAW WIND ENERGY']=np.where(((vel>3)&(vel<25)),calculation_part,0)
def raw_solar_energy_cal(df: DataFrame) -> None:
    df['RAW SOLAR ENERGY'] = df['solarradiation'] * SOLAR_PANEL_EFF * PERF_RATE * (1 - GAMMA * (df['temp_c'] - 25))
raw_wind_energy_cal(df_final)
raw_solar_energy_cal(df_final)


In [214]:
df_final=df_final.set_index(['Full Date','address'])

In [215]:
df_final

,,solarradiation,temp_c,windspeed_ms,RAW WIND ENERGY,RAW SOLAR ENERGY
Full Date,address,,,,,
2025-08-15 00:00:00,izmir,0.0,23.388889,3.039872,13.764549,0.0
2025-08-15 01:00:00,izmir,0.0,26.222222,7.152640,179.306148,0.0
2025-08-15 02:00:00,izmir,0.0,21.777778,3.039872,13.764549,0.0
2025-08-15 03:00:00,izmir,0.0,22.000000,6.795008,153.732609,0.0
2025-08-15 04:00:00,izmir,0.0,29.444444,1.922272,0.000000,0.0
...,...,...,...,...,...,...
2025-11-15 19:00:00,Konya,0.0,8.944444,3.084576,14.380782,0.0
2025-11-15 20:00:00,Konya,0.0,6.944444,2.592832,0.000000,0.0
2025-11-15 21:00:00,Konya,0.0,6.944444,3.084576,14.380782,0.0


In [1]:
def wind_pivot_creator(df: DataFrame) -> DataFrame:
    df_temp=df.reset_index()
    df_filtered=df_temp[df_temp['address'].isin(WIND_CITIES.keys())]
    wind_pivot=df_filtered.pivot(index='Full Date',columns='address',values='RAW WIND ENERGY')
    city_wind_weights=np.array([WIND_CITIES[city] for city in wind_pivot.columns])
    wind_pivot["Total Hourly Energy (RAW)"]=np.dot(wind_pivot.values,city_wind_weights)
    wind_pivot["Total Hourly Energy (RAW)"]=np.true_divide(wind_pivot['Total Hourly Energy (RAW)'].values,1000000)
    return wind_pivot
wind_pivot=wind_pivot_creator(df_final)
wind_pivot

NameError: name 'DataFrame' is not defined

In [217]:
wind_win=wind_pivot['Total Hourly Energy (RAW)'].rolling(window=168,min_periods=1).sum()
best_wind_end=wind_win.idxmax()
best_wind_data=wind_pivot.loc[:best_wind_end].tail(168)
best_avg_wind=best_wind_data["Total Hourly Energy (RAW)"].mean()

worst_wind_end=wind_win.idxmin()
worst_wind_data=wind_pivot.loc[:worst_wind_end].tail(168)
worst_avg_wind=worst_wind_data["Total Hourly Energy (RAW)"].mean()


ValueError: attempt to get argmax of an empty sequence

In [199]:
real_wind_energy=df_energy['Wind Energy'].loc[:best_wind_end].tail(168)
real_avg_wind=real_wind_energy.mean()
WIND_SCALER_BEST=real_avg_wind/best_avg_wind

real_wind_energy=df_energy['Wind Energy'].loc[:worst_wind_end].tail(168)
real_avg_wind=real_wind_energy.mean()
WIND_SCALER_WORST=real_avg_wind/worst_avg_wind

C:\Users\eness\AppData\Local\Temp\ipykernel_28676\3214655592.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  WIND_SCALER_WORST=real_avg_wind/worst_avg_wind


In [200]:
wind_pivot["Total Hourly Energy (EXPECTED - BEST)"]=np.dot(wind_pivot['Total Hourly Energy (RAW)'].values,WIND_SCALER_BEST)
wind_pivot["Total Hourly Energy (EXPECTED - WORST)"]=np.dot(wind_pivot['Total Hourly Energy (RAW)'].values,WIND_SCALER_WORST)
wind_pivot["Total Hourly Energy (REAL)"]=df_energy['Wind Energy'].copy()
wind_pivot

C:\Users\eness\AppData\Local\Temp\ipykernel_28676\4276776265.py:2: RuntimeWarning: invalid value encountered in dot
  wind_pivot["Total Hourly Energy (EXPECTED - WORST)"]=np.dot(wind_pivot['Total Hourly Energy (RAW)'].values,WIND_SCALER_WORST)


address,Balikesir,Canakkale,Izmir,Manisa,Total Hourly Energy (RAW),Total Hourly Energy (EXPECTED - BEST),Total Hourly Energy (EXPECTED - WORST),Total Hourly Energy (REAL)
Full Date,,,,,,,,
2025-08-15 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,6270.54
2025-08-15 01:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,6039.83
2025-08-15 02:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,6061.29
2025-08-15 03:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,6079.25
2025-08-15 04:00:00,35.049412,21.726280,50.734669,28.131268,0.000038,906.148495,inf,6241.11
...,...,...,...,...,...,...,...,...
2025-11-15 19:00:00,84.343997,75.401399,36.757241,43.549835,0.000058,1407.965092,inf,2331.31
2025-11-15 20:00:00,0.000000,0.000000,0.000000,16.231465,0.000003,60.729853,inf,2282.53
2025-11-15 21:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,2107.14
